Between the 6th and the 9th of June 2024, European Parliament elections were held across Europe member states. Particularly in Spain, this elections happened on the 9th of June 2024. The results of them are [publicly known](https://results.elections.europa.eu/en/). Nevertheless, something seeminingly strange happened in the results of the Spanish elections, and it has to deal with the results obtained by the political
party [Junts](https://en.wikipedia.org/wiki/Together_for_Catalonia_(2020)), which has been the **most voted party in several locations outside of Catalonia**, as it can be appreciated in [this applet provided by RTVE](https://www.rtve.es/noticias/20240610/elecciones-europeas-2024-mapa-resultados-calle/16135532.shtml) (an example of this unexpected result can be appreciated in the following figure).

<img src="Junts in Madrid.png" style = "width: 800px; height:400px"/>

Can we say something about this unexpected result? Can this imply that the **election system in Spain has been rigged** in favour of this party? Well, let's see what publicly available data can tell us.

### What is needed to know about Junts

In order to provide some context about this party let me explain what is Junts. *Junts per Catalunya* (together for Catalonia) is a political party in Catalonia (Spain) that stands for catalan independence; that is, Catalonia becoming a country separated from Spain. This party, created in 2020, can be considered the successor of the Catalan European Democratic Party (PDeCAT).
Currently, one of the main and most influential people in the party is [Carles Puigdemont](https://en.wikipedia.org/wiki/Carles_Puigdemont), the former catalan president, who played a key role in the [2017 Catalan ilegal independe referendum](https://en.wikipedia.org/wiki/2017_Catalan_independence_referendum).

***Who is be the target of its politics?***

Well, taking into account that the main focus of the party is making Catalonia to become a country, the main legislation that they will propose will be focused on **the catalan people**. By a similar reasoning, it is safe to assume the main voting electorate of this party to be from Catalonia.

Applying a similar logic once again, it is very likely that **Junts wouldn't get many votes from the other regions of Spain that are not Catalonia**, such as Madrid, Galicia or Andalucía. However, as seen in the previous image, in several locations of the country Junts unexpectedly won or was close to winning.

### Was the election rigged in order to benefit Junts? The theory

In order to contextualize this theory we first have to understand the current distribution of the Spanish Parliament. The current spanish government, composed of a coallition between [PSOE](https://en.wikipedia.org/wiki/Spanish_Socialist_Workers%27_Party), [Sumar](https://en.wikipedia.org/wiki/Sumar_(electoral_platform)) and the consent of smaller political groups, depends on the "yes" of Junts in order to pass many laws (176 diputees to say "yes" in order to achieve absolute majority). In other words, **Junts holds the key of the government**, since it could also allow the opposition to rule the country.

According to the CIS, the Center of Sociological Investigations of Spain, the last report on vote intention polls showed that Junts would [most likely get one eurodeputy](https://www.rtve.es/noticias/20240523/barometro-cis-elecciones-europeas-encuesta-estimacion-voto/16116057.shtml), with a real **chance of getting no representation**, which would mean a loss of two to three eurodeputys with respect to the previous elections in 2019.

So the theory behind an "election rig" becomes easy to understand (this article does not intend the reader to agree with it, just to comprehend its main argument): since the spanish government needs Junts to stay in the government and Junts has a chance of losing representation in the eurochamber, if the government *somehow* doesn't help Junts, Junts will stop helping the government. So, the government must have done something to discretely *boost* Junts results in this elections. This could be an explanation of these strange results, since [Junts almost got two eurodeputys](https://www.rtve.es/noticias/20240609/junts-ue-pierde-dos-eurodiputados-elecciones/16140167.shtml) in the elections.

### What the publicly available data can tell us

Unfortunately, **there is no public dataset on the results of the european elections in spain street-by-street**, despite the fact that several newspapers published interactive maps of the results in such format, such as the one provided in one of the first links of this article. However, what we can see is the number of votes that went to Junts from each municipality of Spain, and from that, count how many votes from Junts came from Catalonia and outside of Catalonia and in how many places of Catalonia did Junts won. In order to do so, we will make use of the publicly available dataset of the [*Area of downloads*](https://infoelectoral.interior.gob.es/es/elecciones-celebradas/area-de-descargas/) of the *Ministerio de Interior* of Spain.

For this article, datasets for the european elections results per municipalities from 2019 and 2024 in Spain have been downloaded and *cleaned* (removed the first rows so that a query could be performed). These files can be found with the project. We will perform several queries of these dataset in order to obtain as much information as possible. We will take into account, that in the 2019 elections the name of the party of study was JUNTS, and for the 2024 elections has been JUNTS UE. For both of these elections we will gather the following information:
* Number of votes coming from Catalonia.
* Number of votes coming from outside of Catalonia.
* Percentage of votes that came from outside of Catalonia.
* List of municipalities outside of Catalonia where the party was first in votes.

Since both of the files that will be queried are of the same format, a previous function that gathers this information will be created.

In [6]:
import pandas as pd

In [8]:
def info(filename, politcal_party, region): 
    #charge excel file to be queried
    xlsx = pd.ExcelFile(filename, engine = 'openpyxl')
    #convert to a readable data structure
    df = pd.read_excel(xlsx)

    #filter the data for municipalities outiside of our region of study
    filtered = df[df['Nombre de Comunidad']!= region]
    filteredReg = df[df['Nombre de Comunidad'] == region]

    xlsx.close()

    totalVotesParty = df[politcal_party].sum()
    totalVotesPartyNOReg = filtered[politcal_party].sum()
    totalVotesPartyReg = filteredReg[politcal_party].sum()
    percentage = 100*totalVotesPartyNOReg / totalVotesParty
    winnerMunicipalities = filtered.loc[
        ((max_votes := filtered.iloc[:,13:].max(axis = 1))!=0) & (filtered[politcal_party] == max_votes),'Nombre de Municipio'
    ]
    return totalVotesPartyReg, totalVotesPartyNOReg, percentage, winnerMunicipalities.tolist()


In [99]:
totalVotes2019, totalVotesOut2019, percentage2019, winnerMunicipalities2019 = info('elections_2019.xlsx','JUNTS', 'Cataluña')
totalVotes2024, totalVotesOut2024, percentage2024, winnerMunicipalities2024 = info('elections_2024.xlsx','JUNTS UE', 'Cataluña')

In [100]:
print("The number of votes of Junts in 2019 was " + str(totalVotes2019) +", while in 2024 was " + str(totalVotes2024) + "."  )
print("The number of votes of Junts from outside of Catalonia in 2019 was " +str(totalVotesOut2019)+ ", while in 2024 was " + str(totalVotesOut2024) + ".")
print("The percentage of votes from the outside of Catalonia in 2019 was " + str(round(percentage2019,2))+ ", and in 2024 was " + str(round(percentage2024,2)) + ".")
print("The party won in 2019 in: " + str(winnerMunicipalities2019) + ", while in 2024 won in " + str(winnerMunicipalities2024) + ".")

The number of votes of Junts in 2019 was 986134, while in 2024 was 431286.
The number of votes of Junts from outside of Catalonia in 2019 was 37051, while in 2024 was 8861.
The percentage of votes from the outside of Catalonia in 2019 was 3.62, and in 2024 was 2.01.
The party won in 2019 in: [], while in 2024 won in [].


## Conclusions

As we have seen in the previous lines, what the data is telling us is that **Junts had a very steep decrease in number of votes of 2024**, less than half of what it had in 2019 and that **the percentage of votes coming from parts outside ouf Catalonia was even lower** than the previous elections. Despite the fact that we cannot tell whether if it did really win in some streets, it is clear that if the government had been to interfere in its results for the better, they could have done a better job, leading to the presented "theory" to lose credibility.

Nevertheless, it would be nice for any public organism of Spain to publish the street by street data of the electoral results, as in the interactive maps that were mentioned in the article, to double check the veracity of the data, while respecting the privacy of the people political choice.

***A much more plausible explanation***

In the first place, the data has provided us insights regarding the real chances of the elections to have been rigged in favour of Junts, which have turned out to be low. Now we can try to provide some context in the nature of these results, and the more plausible is cause is a **human error**. In the following figure we a copy of the location results for the image provided at the begining of the article and its correspondent report of votes, where we can clearly see that the amounts of Junts votes was in reality the amount of votes that PSOE got. This mistake might have happened in several places and might be the main cause of this "unexpected results".

<img src="explanation.JPG" style = "width: 1200px; height:800px"/>